In [5]:
# Dependencies
import pandas as pd

# ZIP CODE DATA

In [33]:
# Store filepath in a variable
Zipcode = "Portland_Zip_and_Neighborhood.csv"

In [34]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zip_df = pd.read_csv(Zipcode, encoding="ISO-8859-1")
zip_df.head()

,Zip_Code,Neighborhood_Name
0,97212,Alameda
1,97217,Arbor Lodge
2,97222,Ardenwald
3,97230,Argay
4,97201,Arlington Heights


# AIRBNB DATA

In [6]:
# Store filepath in a variable
ARBNB = "AB_US_2020.csv"

In [7]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df = pd.read_csv(ARBNB, encoding="ISO-8859-1")

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# Show just the header
ARBNB_df.head()

,ï»¿id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0,Asheville
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.59779,-82.55540,Entire home/apt,470,1,114,7/9/2020,1.03,11,288,Asheville
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,NaN,28801,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,NaN,28806,35.57864,-82.59578,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville
4,160594,Historic Grove Park,769252,Elizabeth,NaN,28801,35.61442,-82.54127,Private room,125,30,58,19/10/15,0.52,1,0,Asheville


In [9]:
column_list = ARBNB_df.columns.values.tolist()

column_list

['ï»¿id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'city']

In [25]:
ARBNB_df.describe()

,ï»¿id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,2.260300e+05,2.260300e+05,226030.000000,226030.000000,226030.000000,2.260300e+05,226030.000000,177428.00000,226030.000000,226030.000000
mean,2.547176e+07,9.352385e+07,35.662829,-103.220662,219.716529,4.525490e+02,34.506530,1.43145,16.698562,159.314856
std,1.317814e+07,9.827422e+07,6.849855,26.222091,570.353609,2.103376e+05,63.602914,1.68321,51.068966,140.179628
min,1.090000e+02,2.300000e+01,18.920990,-159.714900,0.000000,1.000000e+00,0.000000,0.01000,1.000000,0.000000
25%,1.515890e+07,1.399275e+07,32.761783,-118.598115,75.000000,1.000000e+00,1.000000,0.23000,1.000000,0.000000
50%,2.590916e+07,5.138266e+07,37.261125,-97.817200,121.000000,2.000000e+00,8.000000,0.81000,2.000000,140.000000
75%,3.772624e+07,1.497179e+08,40.724038,-76.919323,201.000000,7.000000e+00,39.000000,2.06000,6.000000,311.000000
max,4.556085e+07,3.679176e+08,47.734620,-70.995950,24999.000000,1.000000e+08,966.000000,44.06000,593.000000,365.000000


In [26]:
# Keep the rows that are Portland
ARBNB_Portland_filter=ARBNB_df.loc[(ARBNB_df["city"]=="Portland")]
ARBNB_Portland_filter.head()

,ï»¿id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
166294,16688,"Beautiful condo in downtown Pearl, NW Portland",64840,Ashish,NaN,Pearl,45.52542,-122.68557,Entire home/apt,275,2,83,12/6/2020,2.14,1,353,Portland
166295,29931,Lovely SW Victorian w/Bonus Room and Hot Tub,79786,Ken,NaN,Hayhurst,45.48278,-122.72089,Entire home/apt,200,2,53,31/08/19,0.43,3,36,Portland
166296,37676,Mt. Hood View in the Pearl District,162158,Paul,NaN,Pearl,45.52555,-122.68193,Entire home/apt,100,30,123,11/10/2019,1.01,2,365,Portland
166297,39938,Friendly Guesthouse / 10 Min to Airport & Down...,171371,Sarah,NaN,Woodlawn,45.57428,-122.64594,Private room,29,30,274,18/11/19,4.23,3,89,Portland
166298,41601,Grandpa's Bunkhouse-Backyard Studio,179045,Jean,NaN,Roseway,45.54935,-122.58425,Entire home/apt,89,3,221,5/7/2020,1.83,2,358,Portland


In [63]:
#determine if there are blank data points that need to be dropped
ARBNB_Portland_filter.isna().sum()

ï»¿id                                0
name                                 0
host_id                              0
host_name                            1
neighbourhood_group               4281
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                        481
reviews_per_month                  481
calculated_host_listings_count       0
availability_365                     0
city                                 0
dtype: int64

In [64]:
# drop columns that do not aid in analysis
ARBNB_Portland=ARBNB_Portland_filter.drop(columns=['ï»¿id','name','host_id','host_name',"neighbourhood_group","last_review"])
ARBNB_Portland

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,city
166294,Pearl,45.52542,-122.68557,Entire home/apt,275,2,83,2.14,1,353,Portland
166295,Hayhurst,45.48278,-122.72089,Entire home/apt,200,2,53,0.43,3,36,Portland
166296,Pearl,45.52555,-122.68193,Entire home/apt,100,30,123,1.01,2,365,Portland
166297,Woodlawn,45.57428,-122.64594,Private room,29,30,274,4.23,3,89,Portland
166298,Roseway,45.54935,-122.58425,Entire home/apt,89,3,221,1.83,2,358,Portland
...,...,...,...,...,...,...,...,...,...,...,...
170570,King,45.55745,-122.66108,Private room,56,1,0,NaN,9,16,Portland
170571,Vernon,45.56214,-122.64394,Entire home/apt,250,3,0,NaN,1,18,Portland
170572,Northwest District,45.52729,-122.69451,Entire home/apt,162,3,0,NaN,3,93,Portland
170573,Northwest District,45.53350,-122.69266,Entire home/apt,99,1,0,NaN,1,173,Portland


In [65]:
#determine if there are blank data points that need to be dropped
ARBNB_Portland.isna().sum()

neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
reviews_per_month                 481
calculated_host_listings_count      0
availability_365                    0
city                                0
dtype: int64

In [67]:
#decided to drop nan values from reviews_per_month, could also look at dropping column
ARBNB_Portland_dropna=ARBNB_Portland.dropna()

In [68]:
ARBNB_Portland_dropna.isna().sum()

neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
city                              0
dtype: int64

In [ ]:
#add in zip code and could then drop neighborhood column
# Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
#clustered_df=pd.concat([crypto_df_join,name_df],axis=1)

# ZILLOW DATA

In [11]:
ARBNB_df.dtypes

ï»¿id                               int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
city                               object
dtype: object

In [35]:
# Store filepath in a variable
ZLLW = "portland_housing.csv"

In [36]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ZLLW_df = pd.read_csv(ZLLW, encoding="ISO-8859-1")

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20,50,161,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
# Show just the header
ZLLW_df.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,brokerageName,dateSold,daysOnZillow,description,favoriteCount,...,taxHistory/0/taxPaid,taxHistory/0/time,taxHistory/0/value,taxHistory/0/valueIncreaseRate,url,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zpid
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,NaN,1.620000e+12,25.0,"2860 NE Blossom Hill Rd, Fairview, OR 97024 is...",NaN,...,2989.74,1.590000e+12,172290.0,0.029950,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0,317406.0,5.0,5.0,72258538
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,Harcourts Real Estate Network Group,1.620000e+12,53.0,Cute as a button 3bed/2.5bath home with brand ...,NaN,...,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0,NaN,NaN,NaN,2073593774
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,ERA Freeman & Associates,1.620000e+12,11.0,What a wonderful large home at end of quiet cu...,NaN,...,5178.62,1.590000e+12,282340.0,0.029987,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0,512355.0,5.0,5.0,53959026
3,19309 NE Glisan St,Portland,97230,1.0,3.0,"Premiere Property Group, LLC",1.620000e+12,11.0,NaN,1.0,...,2420.36,1.500000e+12,142330.0,0.029959,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0,NaN,NaN,NaN,176563865
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,NaN,1.620000e+12,14.0,"25-65 NW 20th St, Gresham, OR 97030 is a apart...",NaN,...,2809.61,1.590000e+12,153180.0,0.029989,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0,NaN,NaN,NaN,53819568


In [15]:
column_list2 = ZLLW_df.columns.values.tolist()

column_list2

['ï»¿abbreviatedAddress',
 'address/city',
 'address/zipcode',
 'bathrooms',
 'bedrooms',
 'brokerageName',
 'dateSold',
 'daysOnZillow',
 'description',
 'favoriteCount',
 'homeStatus',
 'homeType',
 'lastSoldPrice',
 'latitude',
 'livingArea',
 'longitude',
 'lotSize',
 'pageViewCount',
 'postingContact/name',
 'price',
 'priceHistory/0/attributeSource/infoString1',
 'priceHistory/0/attributeSource/infoString2',
 'priceHistory/0/attributeSource/infoString3',
 'priceHistory/0/buyerAgent/name',
 'priceHistory/0/buyerAgent/photo/url',
 'priceHistory/0/buyerAgent/profileUrl',
 'priceHistory/0/date',
 'priceHistory/0/event',
 'priceHistory/0/postingIsRental',
 'priceHistory/0/price',
 'priceHistory/0/priceChangeRate',
 'priceHistory/0/pricePerSquareFoot',
 'priceHistory/0/sellerAgent/name',
 'priceHistory/0/sellerAgent/photo/url',
 'priceHistory/0/sellerAgent/profileUrl',
 'priceHistory/0/showCountyLink',
 'priceHistory/0/source',
 'priceHistory/0/time',
 'priceHistory/1/attributeSource/i

In [16]:
ZLLW_df.dtypes

ï»¿abbreviatedAddress     object
address/city              object
address/zipcode            int64
bathrooms                float64
bedrooms                 float64
                          ...   
yearBuilt                float64
zestimate                float64
zestimateHighPercent     float64
zestimateLowPercent      float64
zpid                       int64
Length: 348, dtype: object

In [55]:
ZLLW_df2=ZLLW_df[[
    'ï»¿abbreviatedAddress',
     'address/city',
     'address/zipcode',
     'bathrooms',
     'bedrooms',
     #'brokerageName',
     'dateSold',
     'daysOnZillow',
     #'description',
     #'favoriteCount',
     'homeStatus',
     'homeType',
     'lastSoldPrice',
     'latitude',
     'livingArea',
     'longitude',
     'lotSize',
     'pageViewCount',
     #'postingContact/name',
     'price',
     #'priceHistory/0/attributeSource/infoString1',
     #'priceHistory/0/attributeSource/infoString2',
     #'priceHistory/0/attributeSource/infoString3',
     #'priceHistory/0/buyerAgent/name',
     #'priceHistory/0/buyerAgent/photo/url',
     #'priceHistory/0/buyerAgent/profileUrl',
     'priceHistory/0/date',
     'priceHistory/0/event',
     'priceHistory/0/postingIsRental',
     'priceHistory/0/price',
     'priceHistory/0/priceChangeRate',
     'priceHistory/0/pricePerSquareFoot',
     #'priceHistory/0/sellerAgent/name',
     #'priceHistory/0/sellerAgent/photo/url',
     #'priceHistory/0/sellerAgent/profileUrl',
     #'priceHistory/0/showCountyLink',
     #'priceHistory/0/source',
     #'priceHistory/0/time',
     #'propertyTaxRate',
     'rentZestimate',
     'resoFacts/atAGlanceFacts/0/factValue',
     'resoFacts/basement', 
     'resoFacts/heating/0',
     'resoFacts/atAGlanceFacts/2/factLabel',
     'resoFacts/atAGlanceFacts/2/factValue',
     'resoFacts/cooling/0',
     'resoFacts/atAGlanceFacts/3/factLabel',
     'resoFacts/atAGlanceFacts/3/factValue',
     'taxAssessedValue',
     'taxAssessedYear',
     'taxHistory/0/taxIncreaseRate',
     'taxHistory/0/taxPaid',
     'taxHistory/0/time',
     'taxHistory/0/value',
     'taxHistory/0/valueIncreaseRate',
     'url',
     'yearBuilt',
     'zestimate',
     'zestimateHighPercent',
     'zestimateLowPercent',
     'zpid'
]].copy()
pd.set_option('display.max_columns', None)
ZLLW_df2.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,dateSold,daysOnZillow,homeStatus,homeType,lastSoldPrice,latitude,livingArea,longitude,lotSize,pageViewCount,price,priceHistory/0/date,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/atAGlanceFacts/2/factLabel,resoFacts/atAGlanceFacts/2/factValue,resoFacts/cooling/0,resoFacts/atAGlanceFacts/3/factLabel,resoFacts/atAGlanceFacts/3/factValue,taxAssessedValue,taxAssessedYear,taxHistory/0/taxIncreaseRate,taxHistory/0/taxPaid,taxHistory/0/time,taxHistory/0/value,taxHistory/0/valueIncreaseRate,url,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zpid
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,1.620000e+12,25.0,RECENTLY_SOLD,TOWNHOUSE,315400,45.543571,1806.0,-122.441765,1555.0,47.0,315400,5/21/2021,Sold,False,315400,0.487736,175.0,1950.0,Townhouse,Finished,Forced air,Heating,"Forced air, Gas",NaN,Cooling,NaN,273960.0,2020.0,0.043022,2989.74,1.590000e+12,172290.0,0.029950,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0,317406.0,5.0,5.0,72258538
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,1.620000e+12,53.0,RECENTLY_SOLD,SINGLE_FAMILY,400000,45.547585,1518.0,-122.453224,3484.0,4.0,400000,4/23/2021,Sold,False,400000,0.066667,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Heating,Forced Air,Central Air,Cooling,Central Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0,NaN,NaN,NaN,2073593774
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,512000,45.488228,2724.0,-122.444420,9583.0,153.0,512000,6/4/2021,Sold,False,512000,0.068894,188.0,1995.0,Ranch,Daylight,Forced Air,Heating,"Forced Air, Fireplace(s)",Wall Unit(s),Cooling,Wall Unit(s),397560.0,2020.0,0.026868,5178.62,1.590000e+12,282340.0,0.029987,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0,512355.0,5.0,5.0,53959026
3,19309 NE Glisan St,Portland,97230,1.0,3.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,348000,45.526634,1217.0,-122.464088,13939.0,32.0,348000,6/4/2021,Sold,False,348000,0.025037,286.0,1695.0,Ranch,Crawl Space,Forced Air,Heating,"Forced Air, Fireplace(s)",NaN,Cooling,NaN,269520.0,2017.0,0.045376,2420.36,1.500000e+12,142330.0,0.029959,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0,NaN,NaN,NaN,176563865
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,1.620000e+12,14.0,RECENTLY_SOLD,APARTMENT,510000,45.511238,2400.0,-122.431534,8545.0,2.0,510000,6/1/2021,Sold,False,510000,1.020202,213.0,1395.0,Apartment,NaN,Other,Heating,Other,Central,Cooling,Central,421980.0,2020.0,0.026878,2809.61,1.590000e+12,153180.0,0.029989,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0,NaN,NaN,NaN,53819568


In [56]:
ZLLW_df3=ZLLW_df2.rename(columns={"ï»¿abbreviatedAddress": "Address", "address/city": "City","address/zipcode": "Zipcode"})
ZLLW_df3.head(50)

,Address,City,Zipcode,bathrooms,bedrooms,dateSold,daysOnZillow,homeStatus,homeType,lastSoldPrice,latitude,livingArea,longitude,lotSize,pageViewCount,price,priceHistory/0/date,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/atAGlanceFacts/2/factLabel,resoFacts/atAGlanceFacts/2/factValue,resoFacts/cooling/0,resoFacts/atAGlanceFacts/3/factLabel,resoFacts/atAGlanceFacts/3/factValue,taxAssessedValue,taxAssessedYear,taxHistory/0/taxIncreaseRate,taxHistory/0/taxPaid,taxHistory/0/time,taxHistory/0/value,taxHistory/0/valueIncreaseRate,url,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zpid
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,1.620000e+12,25.0,RECENTLY_SOLD,TOWNHOUSE,315400,45.543571,1806.0,-122.441765,1555.0,47.0,315400,5/21/2021,Sold,False,315400,0.487736,175.0,1950.0,Townhouse,Finished,Forced air,Heating,"Forced air, Gas",NaN,Cooling,NaN,273960.0,2020.0,0.043022,2989.74,1.590000e+12,172290.0,0.029950,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0,317406.0,5.0,5.0,72258538
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,1.620000e+12,53.0,RECENTLY_SOLD,SINGLE_FAMILY,400000,45.547585,1518.0,-122.453224,3484.0,4.0,400000,4/23/2021,Sold,False,400000,0.066667,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Heating,Forced Air,Central Air,Cooling,Central Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0,NaN,NaN,NaN,2073593774
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,512000,45.488228,2724.0,-122.444420,9583.0,153.0,512000,6/4/2021,Sold,False,512000,0.068894,188.0,1995.0,Ranch,Daylight,Forced Air,Heating,"Forced Air, Fireplace(s)",Wall Unit(s),Cooling,Wall Unit(s),397560.0,2020.0,0.026868,5178.62,1.590000e+12,282340.0,0.029987,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0,512355.0,5.0,5.0,53959026
3,19309 NE Glisan St,Portland,97230,1.0,3.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,348000,45.526634,1217.0,-122.464088,13939.0,32.0,348000,6/4/2021,Sold,False,348000,0.025037,286.0,1695.0,Ranch,Crawl Space,Forced Air,Heating,"Forced Air, Fireplace(s)",NaN,Cooling,NaN,269520.0,2017.0,0.045376,2420.36,1.500000e+12,142330.0,0.029959,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0,NaN,NaN,NaN,176563865
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,1.620000e+12,14.0,RECENTLY_SOLD,APARTMENT,510000,45.511238,2400.0,-122.431534,8545.0,2.0,510000,6/1/2021,Sold,False,510000,1.020202,213.0,1395.0,Apartment,NaN,Other,Heating,Other,Central,Cooling,Central,421980.0,2020.0,0.026878,2809.61,1.590000e+12,153180.0,0.029989,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0,NaN,NaN,NaN,53819568
5,1518 SE 12th St,Gresham,97080,2.0,3.0,1.620000e+12,14.0,RECENTLY_SOLD,SINGLE_FAMILY,404200,45.487991,1150.0,-122.416184,7000.0,8.0,404200,6/1/2021,Sold,False,404200,0.976528,351.0,1800.0,SingleFamily,NaN,Forced air,Heating,Forced air,NaN,Cooling,NaN,309260.0,2020.0,0.026862,3232.92,1.590000e+12,176260.0,0.029977,https://www.zillow.com/homedetails/1518-SE-12t...,1967.0,397479.0,6.0,5.0,53963177
6,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,1.620000e+12,25.0,RECENTLY_SOLD,SINGLE_FAMILY,478200,45.498184,2036.0,-122.451332,6969.0,7.0,478200,5/21/2021,Sold,False,478200,0.804528,235.0,2250.0,SingleFamily,Finished,Heat pump,Heating,Heat pump,Central,Cooling,Central,373030.0,2020.0,0.026860,4292.53,1.590000e+12,234030.0,0.029971,https://www.zillow.com/homedetails/110-NW-Will...,1978.0,481241.0,5.0,5.0,53978110
7,4252 SW Lillyben Ave,Gresham,97080,2.0,4.0,1.620000e+12,26.0,RECENTLY_SOLD,SINGLE_FAMILY,530000,45.467598,1947.0,-122.458389,4791.0,69.0,530000,5/20/2021,Sold,False,530000,0.104167,272.0,NaN,Ranch,Crawl Space,Forced Air - 95+%,Heating,"Forced Air - 95+%, Fireplace(s)",Central Air,Cooling,Central Air,426240.0,2020.0,0.024985,4483.12,1.59

In [70]:
ZLLW_df3.isna().sum()

Address                                     0
City                                        0
Zipcode                                     0
bathrooms                                 484
bedrooms                                  770
dateSold                                    0
daysOnZillow                                6
homeStatus                                  0
homeType                                    0
lastSoldPrice                               0
latitude                                   13
livingArea                                465
longitude                                  13
lotSize                                  2890
pageViewCount                           11267
price                                       0
priceHistory/0/date                         0
priceHistory/0/event                        0
priceHistory/0/postingIsRental              0
priceHistory/0/price                        0
priceHistory/0/priceChangeRate              0
priceHistory/0/pricePerSquareFoot 